In [1]:
"""script to convert model into saved model file for tf serving"""

'script to convert model into saved model file for tf serving'

In [2]:
import glob
import os
import numpy as np
import functools
import random
import tensorflow as tf
# import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import losses
import tensorflow.keras.backend as K
from tensorflow.python.keras.callbacks import LambdaCallback
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, \
    Add, Activation, BatchNormalization, Concatenate
from tensorflow.python.keras.models import Model

import segmentation_models as sm

sm.set_framework('tf.keras')

from cb.tbi_cb import TensorBoardImage
from cb.snapshot_cb_builder import SnapshotCallbackBuilder
from cb.sgdr_lr_scheduler import SGDRScheduler


Segmentation Models: using `tf.keras` framework.


In [3]:
MODEL_DIR = "/home/ubuntu/valdata/road_boundary_train/road_satellite/enet"

OUTPUT_SHAPE = (512, 512, 1)
INPUT_SHAPE = (512, 512, 3)

img_shape = (512, 512, 3)
BACKBONE = 'efficientnetb4'
n_classes = 1

In [4]:
# LOSS functions
alpha = K.variable(value=0.1)
alpha._trainable = False

def update_alpha_value(epoch):
    if epoch == 0:
        K.set_value(alpha, 0.1)
        print(f"Setting alpha to = {K.get_value(alpha)}")
    if epoch > 5:
        new_alpha = K.get_value(alpha) + 0.2
        if new_alpha < 0.5:
            K.set_value(alpha, new_alpha)
        else:
            K.set_value(alpha, 0.1)
        print(f"Setting alpha to = {K.get_value(alpha)}")


alpha_update_clb = LambdaCallback(on_epoch_begin=lambda epoch, log: update_alpha_value(epoch))


def segmentation_boundary_loss(y_true, y_pred):
    """
    Using Binary Segmentation mask, generates boundary mask on fly and claculates boundary loss.
    :param y_true:
    :param y_pred:
    :return:
    """
    y_pred_bd = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
    y_true_bd = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
    y_pred_bd = y_pred_bd - (1 - y_pred)
    y_true_bd = y_true_bd - (1 - y_true)

    y_pred_bd_ext = layers.MaxPooling2D((5, 5), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_pred)
    y_true_bd_ext = layers.MaxPooling2D((5, 5), strides=(1, 1), padding='same', input_shape=OUTPUT_SHAPE)(1 - y_true)
    y_pred_bd_ext = y_pred_bd_ext - (1 - y_pred)
    y_true_bd_ext = y_true_bd_ext - (1 - y_true)

    P = K.sum(y_pred_bd * y_true_bd_ext) / K.sum(y_pred_bd) + 1e-7
    R = K.sum(y_true_bd * y_pred_bd_ext) / K.sum(y_true_bd) + 1e-7
    F1_Score = 2 * P * R / (P + R + 1e-7)
    # print(f'Precission: {P.eval()}, Recall: {R.eval()}, F1: {F1_Score.eval()}')
    loss = K.mean(1 - F1_Score)
    # print(f"Loss:{loss.eval()}")
    return loss


def binary_focal_loss(y_true, y_pred):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    """
    alpha = 0.25
    gamma = 2
    pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
    pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
    epsilon = K.epsilon()
    # clip to prevent NaN's and Inf's
    pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
    pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

    return -K.mean(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
           - K.mean((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))


def combined_loss(y_true, y_pred):
#     loss = (1 - alpha) * (binary_focal_loss(y_true, y_pred) + log_cosh_dce_loss(y_true,
#                                                                                 y_pred)) + alpha * segmentation_boundary_loss(y_true, y_pred)
    loss = (1 - alpha) * (losses.binary_crossentropy(y_true, y_pred) + log_cosh_dce_loss(y_true,
                                                                                y_pred)) + alpha * segmentation_boundary_loss(y_true, y_pred)
    return loss


def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss


def log_cosh_dce_loss(y_true, y_pred):
    """
    Implementation suggested in https://arxiv.org/pdf/2006.14822.pdf
    """
    return tf.math.log(tf.math.cosh(dice_loss(y_true, y_pred)))


def dice_coeff(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score


def bce_dice_loss(y_true, y_pred):
    loss = losses.binary_crossentropy(y_true, y_pred) + log_cosh_dce_loss(y_true, y_pred)
    return loss

In [5]:
# MODEL
model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=1, activation='sigmoid', input_shape = (img_shape[0], img_shape[1], 3))

# Segmentation models losses can be combined together by '+' and scaled by integer or float factor
dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss() if n_classes == 1 else sm.losses.CategoricalFocalLoss()
#dice_loss = dice_coef_loss()
#focal_loss = binary_focal_loss()
total_loss = dice_loss + (1 * focal_loss)
dice_loss_metrics = total_loss

# actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
# total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5), sm.metrics.Precision(threshold=0.5),
          sm.metrics.Recall(threshold=0.5), dice_loss_metrics]

model.compile(optimizer='adam', loss=bce_dice_loss, metrics=metrics)

model.summary()

save_model_path = f"{MODEL_DIR}/enetRoadSegV1.h5"

if os.path.exists(save_model_path):
    print("model initialized from saved weights")
    model.load_weights(save_model_path)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 256, 256, 48) 1296        input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 256, 256, 48) 192         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 256, 256, 48) 0           stem_bn[0][0]                    
______________________________________________________________________________________________

In [8]:
def create_tf_serve_model(tf_serving_dir, version):
    """
    :param tf_serving_dir:
    :type tf_serving_dir:
    :param model_path:
    :type model_path:
    :param version:
    :type version:
    :return:
    :rtype:
    """
    
    export_path = os.path.join(tf_serving_dir, str(version))
    print('export_path = {}\n'.format(export_path))
    
    tf.saved_model.save(
        model,
        export_path
    )

In [ ]:
if __name__ == "__main__":
    save_tf_serving_dir = f"{MODEL_DIR}/road_segmentation"
    create_tf_serve_model(save_tf_serving_dir, 1)

export_path = /home/ubuntu/valdata/road_boundary_train/road_satellite/enet/road_segmentation/1

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
